In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3 as lite
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup

## 爬取国家外汇管理局人民币汇率中间价

In [34]:
payload = {'startDate':'2018-10-01',
           'endDate':'2018-11-20',
           'queryYN':'true'}
res = requests.post("http://www.safe.gov.cn/AppStructured/hlw/RMBQuery.do",data=payload)

In [56]:
#res.text

In [118]:
soup = BeautifulSoup(res.text,'html.parser')
print(type(soup.select('#InfoTable')[0]))

<class 'bs4.element.Tag'>


In [115]:
df = pd.read_html(soup.select('#InfoTable')[0].prettify(),header=0) # prettify()将Tag转换为字符串 str bytes

In [116]:
df_data = df[0]

In [117]:
df_data.head()

,日期,美元,欧元,日元,港元,英镑,林吉特,卢布,兰特,韩元,...,挪威克朗,比索,里拉,泰铢,兹罗提,澳元,加元,新西兰元,新加坡元,瑞士法郎
0,2018-11-20,692.80,793.42,6.1610,88.449,890.91,60.423,945.87,202.39,16262.0,...,122.35,293.92,76.706,474.79,54.565,505.41,525.94,473.93,505.34,697.71
1,2018-11-19,692.45,790.46,6.1442,88.423,889.11,60.478,953.11,202.31,16236.0,...,121.88,291.05,77.141,474.72,54.599,506.91,526.87,475.63,504.70,692.79
2,2018-11-16,693.77,785.80,6.1102,88.580,886.34,60.395,949.69,204.73,16261.0,...,122.31,291.91,77.467,474.88,54.566,504.76,526.53,473.61,504.27,689.02
3,2018-11-15,693.92,785.35,6.1084,88.632,901.86,60.434,966.64,206.88,16325.0,...,122.56,294.05,78.719,474.59,54.707,502.00,524.11,471.38,503.34,689.85
4,2018-11-14,694.02,785.21,6.0985,88.613,903.62,60.340,978.37,208.17,16292.0,...,122.16,294.97,78.923,474.45,54.670,501.83,524.79,469.79,503.76,689.96


In [119]:
df_new = df_data.reindex(columns=['日期','美元','欧元','日元'])

In [120]:
df_new.head()

,日期,美元,欧元,日元
0,2018-11-20,692.80,793.42,6.1610
1,2018-11-19,692.45,790.46,6.1442
2,2018-11-16,693.77,785.80,6.1102
3,2018-11-15,693.92,785.35,6.1084
4,2018-11-14,694.02,785.21,6.0985


In [121]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 4 columns):
日期    32 non-null object
美元    32 non-null float64
欧元    32 non-null float64
日元    32 non-null float64
dtypes: float64(3), object(1)
memory usage: 936.0+ bytes


### 存储到数据库

In [122]:
with lite.connect('demo.sqlite') as db:
    df_new.to_sql('demo',con=db,if_exists='append',index=None)

## 创建抓取存储函数

In [151]:
def getCurrency(start,end):
    payload = {'startDate':start,
           'endDate':end,
           'queryYN':'true'}
    res = requests.post("http://www.safe.gov.cn/AppStructured/hlw/RMBQuery.do",data=payload)
    soup = BeautifulSoup(res.text,'html.parser')
    df = pd.read_html(soup.select('#InfoTable')[0].prettify(),header=0)
    df_new = df[0].reindex(columns=['日期','美元','欧元','日元'])
    with lite.connect('rmb.sqlite') as db:
        df_new.to_sql('2018',con=db,if_exists='append',index=None)

In [123]:
now = datetime.now()

### 循环抓取并存储为关系型数据
* 建议使用 SQLite Manager插件浏览

In [152]:
for i in range(1,365,30):
    start_date = (now - timedelta(days = i +30)).strftime('%Y-%m-%d')#生成批量日期，转换为str
    end_date = (now - timedelta(days = i + 1)).strftime('%Y-%m-%d')
    print(start_date,end_date)
    getCurrency(start_date,end_date)

2018-10-21 2018-11-19
2018-09-21 2018-10-20
2018-08-22 2018-09-20
2018-07-23 2018-08-21
2018-06-23 2018-07-22
2018-05-24 2018-06-22
2018-04-24 2018-05-23
2018-03-25 2018-04-23
2018-02-23 2018-03-24
2018-01-24 2018-02-22
2017-12-25 2018-01-23
2017-11-25 2017-12-24
2017-10-26 2017-11-24


# pandas 读取数据库
* 推荐用SQLite可视化工具或插件

In [161]:
# with lite.connect('rmb.sqlite') as con:

#     # read_sql_query和read_sql都能通过SQL语句从数据库文件中获取数据信息
#     df = pd.read_sql_query("SELECT * FROM 2018", con=con)
#     # df = pd.read_sql("SELECT * FROM test_table", con=con)

#     print(df.shape)
#     print(df.dtypes)
#     print(df.head())